# K-nearest neighbors (Movie recommendation system )

---

Imported Libraries

In [1]:
import pandas as pd
import numpy as np

import sqlite3

import json


---

## Step 1: Loading the dataset

We must load the two files and store them in two separate data structures (Pandas DataFrames). On one side, we will have stored the information about the movies and their credits.

In [2]:
movies = pd.read_csv('https://raw.githubusercontent.com/4GeeksAcademy/k-nearest-neighbors-project-tutorial/main/tmdb_5000_movies.csv')
movies.head(3)


,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466


In [3]:
credits = pd.read_csv('https://raw.githubusercontent.com/4GeeksAcademy/k-nearest-neighbors-project-tutorial/main/tmdb_5000_credits.csv')
credits.head(3)

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."


---

## Step 2: Creation of a database

Create a database to store the two DataFrames in separate tables. Then join the two tables with SQL (and integrate it with Python) to generate a third table containing information from both tables unified. The key through which the join can be done is the title of the movie (`title`).

Now, clean the generated table and leave only the following columns:

- `movie_id`  table credits

- `title`   table credits
- `overview`    table movies
- `genres`  table movies
- `keywords`    table movies
- `cast`    table credits
- `crew`    table credits

- ### 2.1 Create a database and save 'movies' and 'credits' in the database

In [4]:
# Create or connect to a database named 'movies_database'

conexion = sqlite3.connect("../data/raw/movies_database.db")

In [5]:
# Save dataframe 'movies' in the database 'movies_database'
# ============================================================================================
movies.to_sql('Movies',
               conexion, # Indicates in which database we are saving the dataframe
                 if_exists = 'replace', # Replace the dataframe if already exists
                   index = False) # Prevents the index column from being saved to the table.


# Save dataframe 'credits' in the database 'movies_database'
# ============================================================================================
credits.to_sql('Credits',
                conexion, # Indicates in which database we are saving the dataframe
                  if_exists = 'replace', # Replace the dataframe if already exists
                    index = False ) # Prevents the index column from being saved to the table.

4803

In [6]:
# It is recommended to log out once you have finished interacting with the database

conexion.close()

---


---

In [2]:
# Create or connect to a database named 'movies_database'

conexion = sqlite3.connect("../data/raw/movies_database.db")

In [12]:
query = '''
CREATE TABLE table3 AS 
SELECT 
    credits.movie_id,
    credits.title, 
    credits.cast, 
    credits.crew,
    movies.overview, 
    movies.genres, 
    movies.keywords 
FROM 
    credits
INNER JOIN
    movies
ON 
    credits.title = movies.original_title;
'''

In [3]:
# Creates a cursor object that is used to execute SQL commands on the database

cursor = conexion.cursor()

In [14]:
# Used to run SQL statements. 
# Here, we drop the table3 if exists
# ================================================================
cursor.execute("DROP TABLE IF EXISTS table3")

In [15]:
# Used to run SQL statements. 
# (query)Create a new combined table with data from `credits` and `movies`
# ================================================================
cursor.execute(query)

In [16]:
# Saves the changes to the database
conexion.commit()

In [4]:
table3 = pd.read_sql("SELECT * FROM table3" ,conexion)

In [18]:
table3.head(3)

,movie_id,title,cast,crew,overview,genres,keywords
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...","In the 22nd century, a paraplegic Marine is di...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de...","Captain Barbossa, long believed to be dead, ha...","[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de...",A cryptic message from Bond’s past sends him o...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name..."


In [5]:
# It is recommended to log out once you have finished interacting with the database
# Closes the connection to the database
conexion.close()

---

## Step 3: Transform the data

Como puedes ver, hay algunas columnas con formato JSON. De cada uno de los JSONs, selecciona el atributo `name` y reemplaza las columnas `genres` y `keywords`. Para la columna `cast`, selecciona los tres primeros nombres.

Las únicas columnas que quedan por modificar son `crew` (equipo) y `overview` (resumen). Para la primera columna, transfórmala para que contenga el nombre del director. Para la segunda, conviértela en una lista.

Una vez hayamos terminado de procesar las columnas y que el modelo de recomendación no se confunda, por ejemplo, entre *Jennifer Aniston* y *Jennifer Conelly*, quitaremos los espacios entre las palabras. Aplica esta función a las columnas `genres`, `cast`, `crew` y `keywords`.

Por último, reduciremos nuestro conjunto de datos combinando todas nuestras columnas convertidas anteriores en una sola columna llamada `tags` (que crearemos). Esta columna ahora tendrá todos los elementos separados por comas y luego las reemplazaremos por espacios en blanco. Debería quedar algo así:

```py
new_df["tags"][0]

>>>>"In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization. Action Adventure Fantasy ScienceFiction cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d SamWorthington ZoeSaldana SigourneyWeaver JamesCameron"
```





columna `crew` nombre del director

columna `overview` convertirla en lista

para columnas `genres` `cast` `crew` y `keywords` quitar espacios entre palabras

In [20]:
table3.head(3)

,movie_id,title,cast,crew,overview,genres,keywords
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...","In the 22nd century, a paraplegic Marine is di...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de...","Captain Barbossa, long believed to be dead, ha...","[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de...",A cryptic message from Bond’s past sends him o...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name..."


In [107]:
modified_table = table3.copy()

---

selecciona el atributo `name` de las columnas `genres` y `keywords` y reemplazalos en las mismas

In [108]:
# Selecciona el atributo 'name' y los reemplaza en las mismas columnas(variable)
# ===============================================================================================
def replace_col(variable): 

    for i in range(len(modified_table[variable])):

        # Converts a JSON-formatted string into a Python object (list or dictionary).
        list = json.loads(modified_table.loc[i, variable]) 

        modified_table.loc[i, variable] = select(list)

    return print(modified_table[variable])


# Elimina espacios
# ===============================================================================================
def select(list):

    global name_list
    name_list = []

    for i in range(len(list)):

        lista = list[i]["name"].replace(" ", "") # Si tiene un espacio lo elimina
    
        name_list.append(lista) # Lo guarda en name_list
    
    
    return to_string(name_list)


# Convierte la lista en un string utilizando como separador un espacio " "
# ===============================================================================================
def to_string(name_list):   

    name_string = " ".join(name_list) 

    return name_string

---

#### `genres`

In [109]:
replace_col("genres")

0       Action Adventure Fantasy ScienceFiction
1                      Adventure Fantasy Action
2                        Action Adventure Crime
3                   Action Crime Drama Thriller
4               Action Adventure ScienceFiction
                         ...                   
4542                      Action Crime Thriller
4543                             Comedy Romance
4544               Comedy Drama Romance TVMovie
4545                                           
4546                                Documentary
Name: genres, Length: 4547, dtype: object


---

#### `keywords`

In [110]:
replace_col("keywords")

0       cultureclash future spacewar spacecolony socie...
1       ocean drugabuse exoticisland eastindiatradingc...
2       spy basedonnovel secretagent sequel mi6 britis...
3       dccomics crimefighter terrorist secretidentity...
4       basedonnovel mars medallion spacetravel prince...
                              ...                        
4542    unitedstates–mexicobarrier legs arms paperknif...
4543                                                     
4544    date loveatfirstsight narration investigation ...
4545                                                     
4546                  obsession camcorder crush dreamgirl
Name: keywords, Length: 4547, dtype: object


---

para la columna cast selecciona los tre primeros `name`